In [10]:
import numpy as np
from cs231n.classifiers.neural_net import TwoLayerNet

In [11]:
DEBUG=True
def dprint(*args):
    if DEBUG:
        print(*args)

In [12]:
input_size = 4
hidden_size = 10
num_classes = 3
num_inputs = 5
def init_toy_model():
    np.random.seed(0)
    return TwoLayerNet(input_size, hidden_size, num_classes, std=1e-1)
def init_toy_data():
    np.random.seed(1)
    X = 10 * np.random.randn(num_inputs, input_size)
    y = np.array([0, 1, 2, 2, 1])
    return X, y
net = init_toy_model()
X, y = init_toy_data()
reg = 0.0
N, D = X.shape
dprint('N, D', N, D)

N, D 5 4


In [13]:
def prod(X, net=net):
    dprint('X', X.shape)
    W1, b1 = net.params['W1'], net.params['b1']
    dprint('W1, b1', W1.shape, b1.shape)
    W2, b2 = net.params['W2'], net.params['b2']
    dprint('W2, b2', W2.shape, b2)
    z = np.dot(X, W1) + b1 # weights multiplied with input data, with bias term b1 added
    dprint('z', z.shape)
    g = np.where(z > 0, z, 0) # ReLU activation function, negative values set to 0.
    dprint('g', g.shape)
    scores = np.dot(g, W2) + b2 # second set of weights multiplied with activated outputs, with second bias term b2 added.
    dprint('scores', scores.shape)

    return scores, (g, W1, b1, W2, b2)

In [14]:
def softmax(_scores, N=N):
    escores = np.exp(_scores)
    probs = escores.T / sum(escores.T)
    return probs.T

In [15]:
def avg_cross_entropy_data_loss(norm_scores):
    y_true_mask = np.zeros_like(norm_scores)
    y_true_mask[np.arange(N), y] = 1
    cel = -np.mean(sum(y_true_mask.T * np.log(norm_scores).T))
    return cel
def L2_regularization_loss(reg, W_list, scaling_factor=0.5):
    norm_sum = 0
    for W in W_list:
        norm_sum += np.sum(W**2) # **2 is a squaring operation; this is like taking the euclidean norm of the weight vectors
    # a scaling factor is often used to "make the derivatives nicer" 
    # ie. have smaller magnitude values for stability of gradient descent
    reg_loss = reg*norm_sum*scaling_factor
    return reg_loss

In [16]:
scores, params = prod(X)
g, W1, b1, W2, b2 = params
probs = softmax(scores)

X (5, 4)
W1, b1 (4, 10) (10,)
W2, b2 (10, 3) [0. 0. 0.]
z (5, 10)
g (5, 10)
scores (5, 3)


In [18]:
loss = avg_cross_entropy_data_loss(probs) + L2_regularization_loss(reg, [W1, W2])
loss

1.2658570521180537

In [26]:
np.sum([[1,2,3], [1,2,3]], axis=0)
# sum([1,2,3])

array([2, 4, 6])

In [19]:
dprob = probs.copy()
dprob[np.arange(N), y] -= 1  # Only for the true y values
# dprob /= N
db2 = sum(dprob) / N
dW2 = np.dot(g.T, dprob) / N
dz = np.dot(dprob, W2.T)
dz[g <= 0] = 0
db1 = sum(dz) / N 
dW1 = np.dot(X.T, dz) / N
dW2 += reg*W2
dW1 += reg*W1


In [27]:
dprob

array([[-0.6355379 ,  0.22911264,  0.40642526],
       [ 0.47590629, -0.82782961,  0.35192332],
       [ 0.43035767,  0.26164229, -0.69199996],
       [ 0.41583127,  0.2983228 , -0.71415407],
       [ 0.36328815, -0.67720061,  0.31391246]])

In [28]:
sum(dprob)

array([ 1.04984549, -0.7159525 , -0.33389299])

In [29]:
np.sum(dprob,axis=0)

array([ 1.04984549, -0.7159525 , -0.33389299])